# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import pickle
import re
import time

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def load_data():
    '''
    Load data from database
    
    Output:
    X:        numpy array (1D) of emergency text messages
    y:        numpy array (2D) of count matrix of labels (i.e. categories of text messages)
    category: list of strings of category names
    
    '''
    engine = create_engine('sqlite:///emergency_messages.db')
    df = pd.read_sql_table('emergency_messages', con=engine)
    
    # define features and label arrays
    X = df.message.values             # turn df column "'message' into numpy.ndarray
    y = df[df.columns[4:]].values     # turn df into numpy.ndarray
    category = list(df.columns[4:])   # turn column names into list
    
    return X, y, category

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    Input:
    text: string. text containing a message.
    
    Output:
    clean_tokens: list of strings. A list of strings containing normalized and lemmatized tokens.
    
    Workflow:
    - Normalize text by converting to lowercase and removing punctuation
    - Tokenize by splitting text up into words
    - Remove Stop words
    - Lemmatize to reduce words to the root or stem form
    '''
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words("english")]
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # remove leading/trailing white space
        clean_tok = tok.strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



In [5]:
# test load function and tokenize function
X, y, category = load_data()
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def model_pipeline():
    '''
    Text processing and machine learning pipeline.
    Create a scikit-learn pipeline to output a ml model that predicts 
    a message classifications for 36 categories of emergency text messages (multi-output classification)
    
    Output: ml model with pipeline 
    '''
    pipeline =  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)), # Convert a collection of text documents to a matrix of token 
            ('tfidf', TfidfTransformer()),          # Transform a count matrix to a normalized tf or tf-idf representation
            ('clf', MultiOutputClassifier(RandomForestClassifier())) # Use the Random Forest algorithm
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# load data
X, y, category = load_data()

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = model_pipeline()

# train pipeline
model.fit(X_train, y_train)

# predict classifiers on test data
y_pred = model.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

- `Accuracy` is defined as the percentage of correct predictions for the test data. It can be calculated by dividing the number of correct predictions by the number of total predictions

In [8]:
# Returns the mean accuracy on the given test data and label
score = model.score(X_test, y_test, sample_weight=None)
print(score)

0.240305149396


- The `precision` is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

- The `recall` is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

- The `F-beta score` can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

- The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.

- The `support` is the number of occurrences of each class in y_true.

In [9]:
print(classification_report(y_test, y_pred, target_names = category, labels=None))

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      6012
               request       0.80      0.42      0.55      1313
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.60      0.66      3255
          medical_help       0.57      0.11      0.18       629
      medical_products       0.73      0.06      0.11       406
     search_and_rescue       0.78      0.03      0.07       202
              security       0.00      0.00      0.00       151
              military       0.59      0.07      0.12       244
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.29      0.44       493
                  food       0.85      0.35      0.50       861
               shelter       0.79      0.26      0.39       715
              clothing       0.88      0.12      0.22       114
                 money       0.43      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The Random Forst model has a high precision and a lower recall. Intuitively, this model shows a high ability of the classifier not to label as positive a sample that is negative. However, it shows a relatively low ability of the classifier to find all the positive samples.    

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Show parameters for the pipline
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ffafd4ced90>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
def find_model():
    '''
    Create a scikit-learn Pipeline with GridSearchCV to output a final model 
    that predicts a message classifications for the 36 categories (multi-output classification)
    
    use GridSearchCV to exhaustive search over specified parameter values for estimator
    Output: cross-validation generator 
    '''
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)) # The number of jobs to run in parallel
                                                                            # -1 means using all processors
    ])

    
    parameters = {
        'vect__max_features': (None, 5000),           # If not None, build a vocabulary that only consider the top 
                                                      # max_features ordered by term frequency across the corpus. 
                                                      # (default=None)
                                                      
        
        'clf__estimator__n_estimators': [100,200]     # number of trees in the forest. The larger the better, 
                                                      # but also the longer it will take to compute. 
                                                      # In addition, note that results will stop getting 
                                                      # significantly better beyond a critical number of trees. 
                                                      # (default=100)
    }
    
    # verbose = 3 to get the detailed realtime progress of the operation
    # instantiate gridsearchcv having cv=2 to speed up computation (2 fold validation, by default it is set to 5)
    cv = GridSearchCV(pipeline, param_grid=parameters,verbose = 3, cv=2, n_jobs=-1, scoring='f1_samples')

    return cv

In [12]:
model_cv = find_model()
model_cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__n_estimators=100, vect__max_features=None .......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=100, vect__max_features=None, score=0.5123915011810609, total= 5.2min
[CV] clf__estimator__n_estimators=100, vect__max_features=None .......


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.3min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=100, vect__max_features=None, score=0.5112809758183677, total= 5.4min
[CV] clf__estimator__n_estimators=100, vect__max_features=5000 .......


...

[CV]  clf__estimator__n_estimators=200, vect__max_features=5000, score=0.5187480653672198, total= 6.8min
[CV] clf__estimator__n_estimators=200, vect__max_features=5000 .......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=200, vect__max_features=5000, score=0.5162867616149125, total= 7.0min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 60.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (None, 5000), 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_samples', verbose=3)

Result of GridSearchCV as table

In [13]:
pd.DataFrame(model_cv.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__n_estimators,param_vect__max_features,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,248.915445,4.446444,67.545905,0.494524,100,None,"{'clf__estimator__n_estimators': 100, 'vect__m...",0.512392,0.511281,0.511836,0.000555,4,0.768082,0.764686,0.766384,0.001698
1,191.310493,1.417540,63.046353,0.048306,100,5000,"{'clf__estimator__n_estimators': 100, 'vect__m...",0.515650,0.513382,0.514516,0.001134,2,0.767403,0.764368,0.765886,0.001517
2,450.412328,6.648899,87.324490,0.876009,200,None,"{'clf__estimator__n_estimators': 200, 'vect__m...",0.513035,0.513281,0.513158,0.000123,3,0.768114,0.764784,0.766449,0.001665
3,335.679039,4.129806,79.109051,0.247143,200,5000,"{'clf__estimator__n_estimators': 200, 'vect__m...",0.518748,0.516287,0.517517,0.001231,1,0.767328,0.764524,0.765926,0.001402


In [14]:
# Best combination of parameters 
model_cv.best_params_

{'clf__estimator__n_estimators': 200, 'vect__max_features': 5000}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
model_tuned = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_features=5000)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200),n_jobs=-1))
    ])

In [16]:
# train pipeline
model_tuned.fit(X_train, y_train)

y_pred = model_tuned.predict(X_test)

Accuracy is defined as the percentage of correct predictions for the test data. It can be calculated by dividing the number of correct predictions by the number of total predictions.

In [17]:
# Returns the mean accuracy on the given test data and label
score = model_tuned.score(X_test, y_test, sample_weight=None)
print(score)

0.281500317864


In [18]:
print(classification_report(y_test, y_pred, target_names = category, labels=None))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      6012
               request       0.82      0.50      0.62      1313
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.70      0.72      3255
          medical_help       0.63      0.14      0.23       629
      medical_products       0.76      0.16      0.26       406
     search_and_rescue       0.66      0.15      0.25       202
              security       0.00      0.00      0.00       151
              military       0.66      0.10      0.18       244
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.53      0.65       493
                  food       0.85      0.70      0.77       861
               shelter       0.80      0.47      0.59       715
              clothing       0.75      0.13      0.22       114
                 money       0.70      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The tuned Random Forsts model has significantly improved on recall compared to the untuned model (0.56 tuned model vs. 0.47 untuned model). Therefore, the tuned model's ability to find all the positive samples is higher. Furthermore, the tuned model shows a marginal improvement on precision as well (0.76 vs. 0.74). The F1-Score improved from 0.53 to 0.60. 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Try other machine learning algorithms AdaBoost developed by Yoav Freund and Robert Schapire (1999)

In [19]:

model_improved = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(),n_jobs=-1))
    ])

In [20]:
# train pipeline

model_improved.fit(X_train, y_train)

y_pred = model_improved.predict(X_test)


In [21]:
print(classification_report(y_test, y_pred, target_names = category, labels=None))

                        precision    recall  f1-score   support

               related       0.80      0.96      0.88      6012
               request       0.77      0.51      0.62      1313
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.59      0.67      3255
          medical_help       0.59      0.25      0.35       629
      medical_products       0.64      0.32      0.42       406
     search_and_rescue       0.62      0.20      0.31       202
              security       0.17      0.03      0.06       151
              military       0.55      0.28      0.37       244
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.66      0.70       493
                  food       0.81      0.66      0.73       861
               shelter       0.74      0.55      0.63       715
              clothing       0.70      0.43      0.53       114
                 money       0.55      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The avg. F1-score of 0.62 for the model using the AdaBoost algorith is higher than agv. F1-score of 0.59 for the tuned model using the Random Forest algorithm. Therefore, we should keep the AdaBoost model.

Let us try to find the best set of parameters for the AdaBoost model. 

Use grid search to find better parameters for the AdaBoost model. 

In [23]:
def find_AdaBoost_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1)) # The number of jobs to run in parallel
                                                                            # -1 means using all processors
    ])

    
    parameters = {
        'vect__max_features': (None, 5000),          # If not None, build a vocabulary that only consider the top 
                                                      # max_features ordered by term frequency across the corpus. 
                                                      # (default=None)
                                                      
        
        'clf__estimator__n_estimators': [50,100]     # n_estimatorsint, default=50 
                                                      
    }

    cv = GridSearchCV(pipeline, param_grid=parameters,verbose = 3, n_jobs=-1, scoring='f1_samples')

    return cv

In [ ]:
best_model = find_AdaBoost_model()

# train pipeline
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)



Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=50, vect__max_features=None, score=0.5314074872066465, total= 2.7min
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.8min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=50, vect__max_features=None, score=0.5264283478394459, total= 2.7min
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


...

[CV]  clf__estimator__n_estimators=50, vect__max_features=5000, score=0.5362578179558622, total= 2.5min
[CV] clf__estimator__n_estimators=100, vect__max_features=None .......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=100, vect__max_features=None, score=0.5260752668940597, total= 3.8min
[CV] clf__estimator__n_estimators=100, vect__max_features=None .......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=100, vect__max_features=None, score=0.5194421055431541, total= 3.8min
[CV] clf__estimator__n_estimators=100, vect__max_features=None .......


Result of GridSearchCV as table

In [ ]:
pd.DataFrame(best_model.cv_results_)

In [ ]:
# Best combination of parameters 
best_model.best_params_

In [ ]:
def build_model():
    '''
    Create a tuned scikit-learn pipeline using the AdaBoost algorithm 
    to output a ml model that predicts a message classifications for 36 categories 
    of emergency text messages (multi-output classification)
    
    Output: ml model with pipeline 
    '''
    model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_features=5000)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50), n_jobs=-1)) 
    ])
    
    return model


In [ ]:
model = build_model()

# train pipeline
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names = category, labels=None))

The avg. F1-score of 0.64 for the tuned AdaBoost model is higher than the F1-score of 0.63 for the untuned model using the AdaBoost algorith and it is higher than agv. F1-score of 0.59 for the tuned model using the Random Forest algorithm. Therefore, we should keep the tuned AdaBoost model. 

### 9. Export your model as a pickle file

In [ ]:
# Save pickle file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.